In [62]:
# import statements

import pandas as pd
import plotly.express as px  # (version 4.7.0)

In [5]:
top_songs = pd.read_csv("top_songs.csv")
top_songs_test = top_songs.groupby('year', as_index=False).mean()

In [6]:
px.line(top_songs_test, x="year", y=["energy", 'danceability'])

In [7]:
# convert dataset so that the audio features are values between 0 and 1
#  top_songs[cols] = sr_df[cols].apply(lambda x: ((x-x.mean()) / (x.std())))

# Sorted Bar Chart Viz

In [8]:
line_copy = top_songs.copy()

# The most popular genres throughout the years
# - With selection feature/slider feature for year
#   - Be able to select different time frames
#   - "Select all" option
# - Display top 10 genres sorted by the count
#Put all the genres into dictionary
genres = dict()
line_copy['artist_genre'] = line_copy['artist_genre'].apply(lambda x : x.replace('[', ''))
line_copy['artist_genre'] = line_copy['artist_genre'].apply(lambda x : x.replace(']', ''))
line_copy['artist_genre'] = line_copy['artist_genre'].apply(lambda x : x.replace("'", ''))
line_copy['artist_genre'] = line_copy['artist_genre'].apply(lambda x : x.split(','))
for lst in line_copy['artist_genre']:
    
    for genre in lst:
        if genre in genres:
            genres[genre] += 1
        else:
            genres[genre] = 1
top_genres_keys = sorted(genres, key=genres.get, reverse=True)[1:10]
top_genres_values = [genres[key] for key in top_genres_keys]

fig = px.bar(x=top_genres_keys, y=top_genres_values, title = 'Top Genres', labels={'x':'Artist Genre', 'y':'Song Count'})
fig

In [9]:
# The artists with the most top songs throughout the years
# - With selection feature/slider feature for year
#   - Be able to select different time frames
#   - "Select all" option
# - Display 10 artists sorted by count of charting songs
line_copy = top_songs.copy()
line_copy['artist_name'] = line_copy['artist_name'].apply(lambda x : x.replace('[', ''))
line_copy['artist_name'] = line_copy['artist_name'].apply(lambda x : x.replace(']', ''))
line_copy['artist_name'] = line_copy['artist_name'].apply(lambda x : x.replace("'", ''))
line_copy['artist_name'] = line_copy['artist_name'].apply(lambda x : x.split(','))

artists_dict = dict()
for artists in line_copy['artist_name']:
    for artist in artists:
        if artist in artists_dict:
            artists_dict[artist] += 1
        else:
            artists_dict[artist] = 1
top_artists_keys = sorted(artists_dict, key=artists_dict.get, reverse=True)[1:10]
top_artists_values = [artists_dict[key] for key in top_artists_keys]

fig = px.bar(x=top_artists_keys, y=top_artists_values, title = 'Top Artists', labels={'x':'Artist Name', 'y':'Song Count'})

fig

## Figure for Genres Exploration over Time

In [29]:
line_copy['artist_genre'] = line_copy['artist_genre'].apply(lambda x : x.replace('[', ''))
line_copy['artist_genre'] = line_copy['artist_genre'].apply(lambda x : x.replace(']', ''))
line_copy['artist_genre'] = line_copy['artist_genre'].apply(lambda x : x.replace("'", ''))
line_copy['artist_genre'] = line_copy['artist_genre'].apply(lambda x : x.split(','))

#Get all the main genres for this year into a dict
genres_dict = dict()
for genres in line_copy['artist_genre']:
    for genre in genres:
        if genre not in genres_dict:
            genres_dict[genre] = genre

genres_dict.keys()

dict_keys(['dance pop', ' pop', ' post-teen pop', 'pop punk', ' punk', ' socal pop punk', 'contemporary country', ' country', ' country dawn', ' country road', ' pop rock', 'glam metal', ' hard rock', ' rock', 'boy band', ' dance pop', 'hip hop', ' hip pop', ' new jack swing', ' pop rap', ' r&b', ' urban contemporary', 'detroit hip hop', ' hip hop', ' rap', ' dance rock', ' europop', ' girl group', ' neo soul', 'bow pop', ' celtic rock', ' new wave pop', 'pop rock', ' southern hip hop', ' st louis rap', 'neo mellow', 'british alternative rock', 'disco house', ' vocal house', ' latin', ' latin pop', ' mexican pop', 'eurodance', 'latin', ' modern salsa', ' salsa', ' tropical', 'bubblegum dance', 'australian pop', ' boy band', 'hip pop', ' quiet storm', ' finnish edm', ' eurodance', ' italian adult pop', ' italo dance', ' electropop', 'beatlesque', ' britpop', ' madchester', ' permanent wave', 'alternative dance', ' big beat', ' electronica', ' trip hop', 'east coast hip hop', 'blues rock

In [158]:
top_songs = pd.read_csv("top_songs.csv")
main_genres_options = ['pop', 'r&b', 'hip hop', 'rap', 'country', 'rock', 'metal', 'house', 'edm', 'indie', 'punk', 'trap', 'reggae', 'latin', 'alternative']

def set_subgenres_options(selected_genre):
    line_copy = top_songs.copy()
    selected_genre = selected_genre[0]

    line_copy['artist_genre'] = line_copy['artist_genre'].apply(lambda x : x.replace('[', ''))
    line_copy['artist_genre'] = line_copy['artist_genre'].apply(lambda x : x.replace(']', ''))
    line_copy['artist_genre'] = line_copy['artist_genre'].apply(lambda x : x.replace("'", ''))
    line_copy['artist_genre'] = line_copy['artist_genre'].apply(lambda x : list(map(str.strip, x.split(','))))

    
    #Create subgenres dict for this main genre
    subgenres_dict = {selected_genre: []}
    
    subgenres_options = dict()
    for genres in line_copy['artist_genre']:
        for genre in genres:
            if selected_genre in genre:

                #Special case for 'rap' selection
                if (selected_genre == 'rap') & ('trap' in genre):
                    continue

                #Add the subgenre to the dict
                subgenres_options[genre] = genre

    return list(subgenres_options.values())

In [161]:
['rap'] in 'rap'set_subgenres_options

TypeError: 'in <string>' requires string as left operand, not list

In [148]:
def set_subgenres_value(available_options):
    return available_options[0]

def set_display_children(selected_subgenre):
    #Main genre
    audio_features = ['energy', 'key','loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'danceability', 'liveness', 'valence', 'tempo', 'duration', 'time_signature']

    indices = [x for x in line_copy['artist_genre'].index if selected_subgenre in line_copy['artist_genre'][x]]
    
    df = line_copy.iloc[indices]
    df = df.groupby('year').mean().reset_index()

    return df

def graph_genre_features(selected_subgenre, genre_feature):
    #Takes in feature to graph
    df = set_display_children(selected_subgenre)
    fig = px.line(df, x='year', y=genre_feature)

    return fig

In [149]:
graph_genre_features('rap', 'loudness')

In [132]:
# subgenres = set_subgenres_options('pop')
# df = set_display_children('rap')
# df = df.loc[df['index'] == 'energy']
# px.bar(df, x='index', y = 'mean')
graph_genre_features('rap', 'danceability')

In [68]:
line_copy = top_songs.copy()

line_copy['artist_genre'] = line_copy['artist_genre'].apply(lambda x : x.replace('[', ''))
line_copy['artist_genre'] = line_copy['artist_genre'].apply(lambda x : x.replace(']', ''))
line_copy['artist_genre'] = line_copy['artist_genre'].apply(lambda x : x.replace("'", ''))
line_copy['artist_genre'] = line_copy['artist_genre'].apply(lambda x : list(map(str.strip, x.split(','))))
line_copy['artist_genre'].apply(lambda x : if 'rap' in x)

SyntaxError: invalid syntax (<ipython-input-68-ba65a9e97036>, line 7)

In [109]:
indices = [x for x in line_copy['artist_genre'].index if 'rap' in line_copy['artist_genre'][x]]
df = line_copy.iloc[indices]
audio_features = ['energy', 'key','loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'danceability', 'liveness', 'valence', 'tempo', 'duration', 'time_signature']
df = df.describe()[audio_features].T.reset_index()
df = df.loc[df['index'] == 'energy']
px.bar(df, x='index', y='mean')


In [99]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,338.0,1029.470414,613.510167,6.000000,469.500000,953.5000,1665.750000,2029.000
year,338.0,2009.710059,6.031959,2000.000000,2004.000000,2009.0000,2016.000000,2019.000
danceability,338.0,0.733379,0.129403,0.325000,0.648000,0.7375,0.836500,0.969
energy,338.0,0.667905,0.140589,0.329000,0.566500,0.6705,0.768750,0.974
key,338.0,5.576923,3.778158,0.000000,1.000000,6.0000,9.000000,11.000
loudness,338.0,-5.931109,1.910715,-13.203000,-7.057750,-5.8090,-4.750000,-1.190
mode,338.0,0.565089,0.496480,0.000000,0.000000,1.0000,1.000000,1.000
speechiness,338.0,0.188489,0.120047,0.026900,0.080325,0.1725,0.275000,0.530
acousticness,338.0,0.111285,0.129171,0.000107,0.022475,0.0600,0.153000,0.688
instrumentalness,338.0,0.001718,0.017697,0.000000,0.000000,0.0000,0.000002,0.282
